In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [106]:
#Check g_key
#g_key

In [3]:
#Read weather csv into a DataFrame
weather_path = os.path.join("Output", "cities.csv")
weather_data = pd.read_csv(weather_path)
weather_data.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bluff,-46.60,168.33,39.00,83,100,3.00,NZ,1595272511
1,1,Wagga Wagga,-35.12,147.37,33.80,100,90,3.89,AU,1595272511
2,2,Arraial do Cabo,-22.97,-42.03,80.60,78,0,13.87,BR,1595272511
3,3,Yuzhnyy,53.25,83.69,71.60,64,40,4.47,RU,1595272511
4,4,Cherskiy,68.75,161.30,67.57,45,86,4.81,RU,1595272511


In [5]:
 # Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [7]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Latitude", "Longitude"]].astype(float)

In [20]:
#Define Humidity
humidity = weather_data["Humidity"].astype(float)

#Set Map figure orientation, then add heat layer for Humidity
fig = gmaps.figure(zoom_level=2, center=(0,0))
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=100, 
                                 point_radius = 3)
fig.add_layer(heat_layer)

#Show the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [89]:
#My ideal vacation is cold and rainy (cozy, no other tourists, etc.)
#Create a data frame using the following weather criteria
ideal_weather = weather_data.loc[(weather_data["Cloudiness"]>75)&
                                 (weather_data["Max Temperature"]<70)&
                                 (weather_data["Wind Speed"]<5)&
                                 (weather_data["Humidity"]<70)]
ideal_weather

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Cherskiy,68.75,161.30,67.57,45,86,4.81,RU,1595272511
150,150,Saskylakh,71.92,114.08,50.76,66,83,4.97,RU,1595272551
165,165,Port Lincoln,-34.73,135.87,52.72,64,79,4.99,AU,1595272555
230,230,Kaitangata,-46.28,169.85,50.00,65,100,1.99,NZ,1595272502
247,247,Goma,-1.68,29.22,66.72,65,82,1.12,CD,1595272577
291,291,Taltal,-25.40,-70.48,64.20,58,100,3.67,CL,1595272590
358,358,Vangaindrano,-23.35,47.60,65.41,66,100,2.13,MG,1595272609
575,575,Otta,61.77,9.54,55.92,67,100,3.49,NO,1595272659


In [95]:
#Create Hotel Name column (string)
ideal_weather["Hotel Name"]= ""
ideal_weather

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,Cherskiy,68.75,161.30,67.57,45,86,4.81,RU,1595272511,
150,150,Saskylakh,71.92,114.08,50.76,66,83,4.97,RU,1595272551,
165,165,Port Lincoln,-34.73,135.87,52.72,64,79,4.99,AU,1595272555,
230,230,Kaitangata,-46.28,169.85,50.00,65,100,1.99,NZ,1595272502,
247,247,Goma,-1.68,29.22,66.72,65,82,1.12,CD,1595272577,
291,291,Taltal,-25.40,-70.48,64.20,58,100,3.67,CL,1595272590,
358,358,Vangaindrano,-23.35,47.60,65.41,66,100,2.13,MG,1595272609,
575,575,Otta,61.77,9.54,55.92,67,100,3.49,NO,1595272659,


In [96]:
#Define search parameters
target_type = "lodging"
radius = 5000

params = {"key": g_key,
          "types": target_type, 
          "radius": radius}

    
#For loop, using cities to find hotels, store in hotel list
hotel_list = []

for index,row in ideal_weather.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"
    
#API Call for Hotel names   
    hotels = requests.get(base_url, params=params)
#Convert to JSON
    hotels = hotels.json()
    
#Hotel dictionary for storing hotel name information
    hotel = {}

    try:
        hotel["Hotel Name"] = hotels["results"][0]["name"]
        ideal_weather.loc[index, "Hotel Name"]= hotels["results"][0]["name"]
        hotel_list.append(hotel)
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
ideal_weather

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.


,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,Cherskiy,68.75,161.30,67.57,45,86,4.81,RU,1595272511,Gostinitsa
150,150,Saskylakh,71.92,114.08,50.76,66,83,4.97,RU,1595272551,
165,165,Port Lincoln,-34.73,135.87,52.72,64,79,4.99,AU,1595272555,Port Lincoln Hotel
230,230,Kaitangata,-46.28,169.85,50.00,65,100,1.99,NZ,1595272502,Kaitangata Motor Camp
247,247,Goma,-1.68,29.22,66.72,65,82,1.12,CD,1595272577,"Goma, CENTRE D'ACCUEIL CARITAS"
291,291,Taltal,-25.40,-70.48,64.20,58,100,3.67,CL,1595272590,Residencial Karime
358,358,Vangaindrano,-23.35,47.60,65.41,66,100,2.13,MG,1595272609,Hotel Fanilo
575,575,Otta,61.77,9.54,55.92,67,100,3.49,NO,1595272659,Thon Hotel Otta


In [103]:
#Hotel, City, Country information for markers
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]
locations1 = ideal_weather[["Latitude", "Longitude"]]

In [105]:
fig1 = gmaps.figure(zoom_level=2, center=(0,0))
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations1,info_box_content=hotel_info)

# Add the layer to the map
fig1.add_layer(markers)
fig1.add_layer(heat_layer)
fig1

Figure(layout=FigureLayout(height='420px'))